In [10]:
import importlib
import imageio.v2 as imageio
from patient import patient
import glob
import cupy as cp

p = patient.patient(
    name="4404151",
    # name="4409573",
    # name="4411423",
    # name="4411557",
    do={'ct': False, 'drr': True, 'posdrr': False, 'fbp': True,
        'posfbp': False, 'resize': False},
    skip_done=False
)


# imgs = [cp.asnumpy(img) for img in p.drr.img]
# imageio.mimsave("x.gif", imgs)
# p.generate_fbp(load_all=False)

ValueError: 

In [ ]:
i = 2

pos = 200

imageio.imsave("ct.png", cp.asnumpy(p.ct.img[pos]))
imageio.imsave("fbp.png",
cp.asnumpy(p.get_equiv_fbp(pos)))

# imgct = imageio.imread("ct.png").astype("float32")
# imgrec = imageio.imread(f"fbp.png").astype("float32")

# imageio.imsave(f"diff{i}.png", imgct - imgrec)
%slack

In [3]:
imgs = [cp.asnumpy(img) for img in p.drr.img]

imageio.mimsave("x.gif", imgs)

In [3]:
%slack

In [ ]:
import importlib
import imageio.v2 as imageio
from patient import patient
import glob

poss = [50, 100, 150, 200]
vols = ["4404147", "4404366"]

for idx, vol in enumerate(vols):
    p = patient.patient(name=vol)
    for pos in poss:
        imageio.imsave(f"figures/ct_patient{idx}_{pos}.png",
                       cp.asnumpy(p.ct.img[pos]))

# imageio.mimsave(f"pics/drr.gif", p.drr.img)
%slack